# **Clean and Prep ADA Race Results v2 - SQLite**

Cleans and standardizes 5K@ADA race results by removing duplicates, translating gender values into English, and updating a local SQLite database. Participant names are used briefly for deduplication and then discarded to maintain privacy.

Run on Python 3.13 | No errors | No warnings

In [1]:
# Import packages

# For data manipulation
import pandas as pd

# for displaying and modifying the working directory
import os as os

# For working with SQLite databases
import sqlite3

In [2]:
# Load the SQL Magic extension
%load_ext sql

In [3]:
# Create connection to the SQLite database
%sql sqlite:///ADARaceResults.db

Connecting to 'sqlite:///ADARaceResults.db'

In [4]:
# Define custom NA values, excluding 'NA' to make sure that Country Code NA (Namibia) is not interpreted as missing data
custom_na_values = ['N/A', 'NaN', 'null', '']

In [5]:
#Load specific columns, ignore the default NA values and use the custom ones
df0 = pd.read_csv("ADA_race_results.csv", usecols=['Name', 'Country', 'Time', 'Enrollment', 'Gender'], keep_default_na=False, na_values=custom_na_values)

# Display the first 5 rows of the dataframe
df0.head()

,Name,Gender,Country,Time,Enrollment
0,Yousif Alghaderi,Male,BH,00:14:45,Run / Walk
1,Yousif Alghaderi,Male,BH,00:14:45,Run / Walk
2,Mohamed Alsharkaoy,Male,EG,00:14:45,Run / Walk
3,Muntasir Ahmad Abdul Kareem Alzghoul,Male,JO,00:14:45,Run / Walk
4,Muntasir Ahmad Abdul Kareem Alzghoul,Male,JO,00:14:45,Handcycle / Wheelchair


In [6]:
# Display basic information about the data 
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7048 entries, 0 to 7047
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        7048 non-null   object
 1   Gender      7048 non-null   object
 2   Country     7048 non-null   object
 3   Time        7048 non-null   object
 4   Enrollment  7048 non-null   object
dtypes: object(5)
memory usage: 275.4+ KB


In [7]:
# Display the number of missing values in each column
df0.isnull().sum()

Name          0
Gender        0
Country       0
Time          0
Enrollment    0
dtype: int64

In [8]:
# Verify the duplicates have been removed
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7048 entries, 0 to 7047
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        7048 non-null   object
 1   Gender      7048 non-null   object
 2   Country     7048 non-null   object
 3   Time        7048 non-null   object
 4   Enrollment  7048 non-null   object
dtypes: object(5)
memory usage: 275.4+ KB


In [9]:
# Add a column for the year
df0['Year'] = 2025

In [10]:
# Add a column for the race name
df0['Race'] = 'ADA'

In [11]:
# Update Gender values
gender_map = {
    'Masculino': 'Male',
    'Femenino': 'Female',
    'Desconocido': 'Unknown',
    '男性': 'Male',
    '女性': 'Female',
    'ç”·æ€§': 'Male',     # Unicode mishandling
    'å¥³æ€§': 'Female',   # Unicode mishandling
    '分からない': 'Unknown',
}
df0['Gender'] = df0['Gender'].replace(gender_map)

In [12]:
# Verify the changes
df0.head()

,Name,Gender,Country,Time,Enrollment,Year,Race
0,Yousif Alghaderi,Male,BH,00:14:45,Run / Walk,2025,ADA
1,Yousif Alghaderi,Male,BH,00:14:45,Run / Walk,2025,ADA
2,Mohamed Alsharkaoy,Male,EG,00:14:45,Run / Walk,2025,ADA
3,Muntasir Ahmad Abdul Kareem Alzghoul,Male,JO,00:14:45,Run / Walk,2025,ADA
4,Muntasir Ahmad Abdul Kareem Alzghoul,Male,JO,00:14:45,Handcycle / Wheelchair,2025,ADA


In [13]:
# Rename Country to Country Code
df0 = df0.rename(columns={'Country': 'Country Code'})

In [14]:
# Rename Gender column to Age Group
df0 = df0.rename(columns={'Gender': 'Age Group'})

In [15]:
# Change the column order
df0 = df0[['Race', 'Year', 'Country Code', 'Age Group', 'Time', 'Enrollment', 'Name']]

In [16]:
# Verify the changes
df0.head()

,Race,Year,Country Code,Age Group,Time,Enrollment,Name
0,ADA,2025,BH,Male,00:14:45,Run / Walk,Yousif Alghaderi
1,ADA,2025,BH,Male,00:14:45,Run / Walk,Yousif Alghaderi
2,ADA,2025,EG,Male,00:14:45,Run / Walk,Mohamed Alsharkaoy
3,ADA,2025,JO,Male,00:14:45,Run / Walk,Muntasir Ahmad Abdul Kareem Alzghoul
4,ADA,2025,JO,Male,00:14:45,Handcycle / Wheelchair,Muntasir Ahmad Abdul Kareem Alzghoul


In [17]:
# Display a list of unique values for Age Group
df0['Age Group'].unique()

array(['Male', 'Female', 'Non-binary', 'Unknown'], dtype=object)

In [18]:
# Load the dataframe into the SQLite database
df0.to_sql("ADA_RACE_DATA", con=sqlite3.connect('ADARaceResults.db'), if_exists='append', index=False)

7048

In [19]:
%%sql
DELETE FROM ADA_RACE_DATA
WHERE "Age Group" IN (
    '女性',
    '男性',
    'ç”·æ€§',
    'å¥³æ€§',
    '分からない'
);

Running query in 'sqlite:///ADARaceResults.db'

++
||
++
++

In [20]:
# Display column names and data types for the ADA_RACE_DATA table
%sql PRAGMA table_info(ADA_RACE_DATA)

Running query in 'sqlite:///ADARaceResults.db'

cid,name,type,notnull,dflt_value,pk
0,Race,TEXT,0,None,0
1,Year,INTEGER,0,None,0
2,Country Code,TEXT,0,None,0
3,Age Group,TEXT,0,None,0
4,Time,TEXT,0,None,0
5,Enrollment,TEXT,0,None,0
6,Name,TEXT,0,None,0


In [21]:
# Get the record count for ADA_RACE_DATA
%sql SELECT COUNT(*) FROM ADA_RACE_DATA

Running query in 'sqlite:///ADARaceResults.db'

COUNT(*)
19616


In [22]:
%%sql
-- Find the total count of duplicate rows in the ADA_RACE_DATA table
SELECT SUM(duplicate_count - 1) AS total_duplicates
FROM (
    SELECT COUNT(*) AS duplicate_count
    FROM ADA_RACE_DATA
    GROUP BY "Race", "Year", "Country Code", "Age Group", "Time", "Enrollment", "Name"
    HAVING COUNT(*) > 1
) AS duplicates;

Running query in 'sqlite:///ADARaceResults.db'

total_duplicates
7039


In [23]:
%%sql
-- Delete duplicate rows in the ADA_RACE_DATA table
DELETE FROM ADA_RACE_DATA
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM ADA_RACE_DATA
    GROUP BY "Race", "Year", "Country Code", "Age Group", "Time", "Enrollment", "Name"
);

Running query in 'sqlite:///ADARaceResults.db'

7039 rows affected.

++
||
++
++

In [24]:
# Get the record count for ADA_RACE_DATA
%sql SELECT COUNT(*) FROM ADA_RACE_DATA

Running query in 'sqlite:///ADARaceResults.db'

COUNT(*)
12577


In [25]:
# Retrieve all data from the ADA_RACE_DATA table
df = %sql SELECT * FROM ADA_RACE_DATA

# Convert the ResultSet to a DataFrame
df1 = df.DataFrame()

Running query in 'sqlite:///ADARaceResults.db'

In [26]:
# Display the first 5 rows of the dataframe (df1)
df1.head()

,Race,Year,Country Code,Age Group,Time,Enrollment,Name
0,ADA,2024,TD,30 - 34,0:14:45,Run / Walk,A
1,ADA,2024,CO,20 - 24,0:14:45,Run / Walk,B
2,ADA,2024,ZA,35 - 39,0:14:45,Run / Walk,C
3,ADA,2024,US,40 - 44,0:14:45,Run / Walk,D
4,ADA,2024,VE,50 - 54,0:14:45,Run / Walk,E


In [27]:
# Display basic information about the data
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12577 entries, 0 to 12576
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Race          12577 non-null  object
 1   Year          12577 non-null  int64 
 2   Country Code  12577 non-null  object
 3   Age Group     12577 non-null  object
 4   Time          12577 non-null  object
 5   Enrollment    12577 non-null  object
 6   Name          12577 non-null  object
dtypes: int64(1), object(6)
memory usage: 687.9+ KB


In [28]:
# Drop the Name column
df1 = df1.drop(columns=['Name'])

In [29]:
df1 = df1.sort_values(by=["Race", "Year", "Time"])

In [30]:
# Verify the change
df1.head()

,Race,Year,Country Code,Age Group,Time,Enrollment
0,ADA,2024,TD,30 - 34,0:14:45,Run / Walk
1,ADA,2024,CO,20 - 24,0:14:45,Run / Walk
2,ADA,2024,ZA,35 - 39,0:14:45,Run / Walk
3,ADA,2024,US,40 - 44,0:14:45,Run / Walk
4,ADA,2024,VE,50 - 54,0:14:45,Run / Walk


In [31]:
# Save the cleaned data to a new csv file
df1.to_csv('ADA Race Data for Visualization.csv', index=False)